<a href="https://colab.research.google.com/github/yashyaks/ML-playground/blob/main/ACL/ACL_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
df.head()

,review,sentiment
0,This film should have never been made. Honestl...,negative
1,This movie was bad from the start. The only pu...,negative
2,"God, I never felt so insulted in my whole life...",negative
3,Not being a fan of the Coen Brothers or George...,positive
4,The movie Andaz Apna Apna in my books is the t...,positive


In [ ]:
df['review'][1]

"This movie was bad from the start. The only purpose of the movie was that Angela wanted to get a high body count. The acting was horrible. The killings were acted out very badly. Like when Ally got stuffed down that toilet I guess it was in the abandoned cabin. But when the end of the movie comes and Molly and the other guy are in the cabin you see Ally so Angela must have gone in to get her. The part that really got me was when the black girl and Angela were in the cabin and Angela took the guitar string and chocked her. One it was horrible acting and two why wouldn't you just turn around and punch the bitch?!?!? Then when Molly is getting chased by Angela if you have the neigh why not just turn around and stab her??? So stupid. This movie sucked..."

In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
positive,5016
negative,4984


In [ ]:
df.isnull().sum()

,0
review,0
sentiment,0


In [ ]:
df.duplicated().sum()

24

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [ ]:
df['review'] = df['review'].apply(remove_tags)

In [ ]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=69)

### Using BoW (Bag of Words)



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [ ]:
X_train_bow.shape

(7980, 48245)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8512024048096193

### Using TfIdf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

In [ ]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

0.8396793587174348

### Word2Vec


In [ ]:
import gensim

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5852242, 6186135)

In [ ]:
len(model.wv.index_to_key)

32012

In [ ]:
def document_vector(doc):
    tokens = simple_preprocess(doc)
    tokens = [word for word in tokens if word in model.wv.index_to_key]
    if len(tokens) == 0:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[tokens], axis=0)

X = np.array([document_vector(doc) for doc in df['review']])
y = df['sentiment']
# y = encoder.fit_transform(y)

# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.77
